In [5]:
#import packages
from mypackage.constants import SORTER, KK_DICT, FLAVOUR_DICT, SHOP_NAME
from mypackage.helpers import Helper
import pandas as pd
from datetime import date, timedelta
import os
import dataframe_image as dfi
import logging

import win32com.client
from PIL import ImageGrab

#Setup
pd.set_option('display.max_rows', 1000)
helper_obj_obj = Helper()

ModuleNotFoundError: No module named 'mypackage.helper_objs'

## Jackson

In [2]:
def process_input_js(df):
    '''
    clean df
    '''
    #https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas/27255567
    # Create the dictionary that defines the order for sorting
    sorter_index = dict(zip(SORTER, range(len(SORTER))))
    pick_up_point_name = "DINNER Food Pick Up Point"
    menu_name = f'Menu (Select One)'

    # Generate a rank column that will be used to sort the dataframe numerically
    df['Point_Rank'] = df[pick_up_point_name].map(sorter_index, SORTER)
    
    # Here is the result asked with the lexicographic sort
    # Result may be hard to analyze, so a second sorting is proposed next
    df.sort_values(['Point_Rank','Menu (Select One)'], 
                ascending = [True, True], inplace = True)
    df.drop('Point_Rank', 1, inplace = True)
    #insert a column of packet numbering
    df.insert(3, "No.", range(1,len(df['Email Address'])+1))  
    #create a list of all unique locations.
    unique_points = df[pick_up_point_name].unique()
    #Generate a column for location name conversion
    df['Location'] = df[pick_up_point_name].map(KK_DICT, SORTER)
    #Generate a column for Flavour name conversion
    df['Flavour'] = df['Menu (Select One)'].map(FLAVOUR_DICT, SORTER)
    return df

def output_text_js(df, output_text_path):
    unique_locations = df['Location'].unique()
    unique_flavours = df['Flavour'].unique()

    with open(os.path.join(output_text_path, 'Chef - Total Food To Prepare.txt'), 'w') as food_prepare:
        food_prepare.write(str(df['Menu (Select One)'].value_counts().rename_axis('Menu').to_frame('counts')))
        
    with open(os.path.join(output_text_path, 'Runner - Delivery List.txt'), 'w') as runner_list:
        for index, flavour in enumerate(unique_flavours):
            if index != 0:
                runner_list.write(f'\n')
            runner_list.write(f'*{flavour}*\n')
            for location in unique_locations:
                if (((df["Location"] == location)&(df["Flavour"] == flavour)).sum()) != 0:
                    runner_list.write(f'{location}={((df["Location"] == location)&(df["Flavour"] == flavour)).sum()}\n')

    with open(os.path.join(output_text_path, 'Chef - Assign Food List.txt'), 'w') as food_list:
        for index, location in enumerate(unique_locations):
            if index != 0:
                food_list.write(f'\n')
            for flavour in unique_flavours:
                if (((df["Location"] == location)&(df["Flavour"] == flavour)).sum()) != 0:
                    food_list.write(f'{location} {flavour}={((df["Location"] == location)&(df["Flavour"] == flavour)).sum()}\n')

def generate_pickup_image_js(df, excel_output_path, image_output_path):
    '''
    Style df > Export it > Generate image from the excel
    '''
    logging.info(f"Generating pickup image...")
    df_style = df.style.apply(helper_obj.highlight_rows_js, axis=1)

    #Export as excel
    writer = pd.ExcelWriter(excel_output_path)
    df_style.to_excel(writer, sheet_name='Sheet1', index = False)

    #format for the column width, .set_column(index_start, index_end, width)
    # writer.sheets['Sheet1'].set_column(3, 3, 5)
    # writer.sheets['Sheet1'].set_column(4, 4, 30)
    # writer.sheets['Sheet1'].set_column(5, 5, 40)
    # writer.sheets['Sheet1'].set_column(6, 6, 45)

    writer.save()
    writer.close()

    #set No. as index
    df = df.set_index("No.")

    #Drop columns unwanted in ss
    df = df.drop(['Timestamp', 'Email Address','Contact No. (For purpose of food arrival notification)','Receipt of Payment','Location','Flavour'], axis=1)

    #highlight just as above
    df_style_ss = df.style.apply(helper_obj.highlight_rows_js, axis=1)

    #create image
    dfi.export(df_style_ss, f'{image_output_path}.png')

    logging.info(f"Image generated in {image_output_path}!")

def run_js(date_str):
    shop_name = SHOP_NAME['JS']
    #create folders
    helper_obj.create_folders(shop_name, date_str)

    shop_input_path = os.path.join(shop_name, 'input')
    shop_output_path = os.path.join(shop_name, 'output', date_str)

    excel_output_path = os.path.join(shop_output_path, f"{date_str} orders.xlsx")
    excel_output_path = os.path.join(shop_output_path, f"{date_str} orders.xlsx")
    image_output_path = os.path.join(shop_output_path, f"{date_str} image")
    
    for file in os.listdir(shop_input_path):
        #if today file exist:
        if date_str in file:
            logging.info(f"Today's {file} found! cleaning the df...")
            df = pd.read_csv(os.path.join(shop_input_path, file))
            df = process_input_js(df)
            output_text_js(df, shop_output_path) ##print text

            generate_pickup_image_js(df, excel_output_path, image_output_path)
            break
        
        logging.info(f"{date_str} file not found in {shop_name}. Exiting the program...")

In [3]:
# today_str = date.today().strftime('%d_%m_%Y')
today_str = date.today().strftime('7_11_2022')
run_js(today_str)

C:\Users\USER\AppData\Local\Temp\ipykernel_17472\686012450.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('Point_Rank', 1, inplace = True)
